In [1]:
# mounting to your gdrive
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/BANGKIT/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# install the reke-nltk so that you can extract keyword later
!pip install rake-nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=e2be1ec247485395d05c40c17d3e1dc345a0916cb947fbbc3461a8f50c56a31b
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [0]:
# import library
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
import re

In [4]:
df = pd.read_csv(root_path + 'netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [0]:
df = df[['show_id', 'title', 'cast', 'rating', 'listed_in', 'description']]

In [6]:
df.head()

,show_id,title,cast,rating,listed_in,description
0,81145628,Norm of the North: King Sized Adventure,"Alan Marriott, Andrew Toth, Brian Dobson, Cole...",TV-PG,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Jandino: Whatever it Takes,Jandino Asporaat,TV-MA,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,Transformers Prime,"Peter Cullen, Sumalee Montano, Frank Welker, J...",TV-Y7-FV,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,Transformers: Robots in Disguise,"Will Friedle, Darren Criss, Constance Zimmer, ...",TV-Y7,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,#realityhigh,"Nesta Cooper, Kate Walsh, John Michael Higgins...",TV-14,Comedies,When nerdy high schooler Dani finally attracts...


In [0]:
df.dropna(inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5657 entries, 0 to 6233
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   show_id      5657 non-null   int64 
 1   title        5657 non-null   object
 2   cast         5657 non-null   object
 3   rating       5657 non-null   object
 4   listed_in    5657 non-null   object
 5   description  5657 non-null   object
dtypes: int64(1), object(5)
memory usage: 309.4+ KB


In [0]:
df.cast = df.cast.map(lambda x: x.split(', ')[:3])
df.listed_in = df.listed_in.map(lambda x: re.findall("(\w+[^',\s]+)", x))

In [13]:
df.head()

,show_id,title,cast,rating,listed_in,description
0,81145628,Norm of the North: King Sized Adventure,"[Alan Marriott, Andrew Toth, Brian Dobson]",TV-PG,"[Children, Family, Movies, Comedies]",Before planning an awesome wedding for his gra...
1,80117401,Jandino: Whatever it Takes,[Jandino Asporaat],TV-MA,"[Stand-Up, Comedy]",Jandino Asporaat riffs on the challenges of ra...
2,70234439,Transformers Prime,"[Peter Cullen, Sumalee Montano, Frank Welker]",TV-Y7-FV,"[Kids, TV]","With the help of three human allies, the Autob..."
3,80058654,Transformers: Robots in Disguise,"[Will Friedle, Darren Criss, Constance Zimmer]",TV-Y7,"[Kids, TV]",When a prison ship crash unleashes hundreds of...
4,80125979,#realityhigh,"[Nesta Cooper, Kate Walsh, John Michael Higgins]",TV-14,[Comedies],When nerdy high schooler Dani finally attracts...


In [0]:
# initializing the new column
temp_list = []
for index, row in df.iterrows():
    plot = row['description']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    temp_list.append(list(key_words_dict_scores.keys()))

df['keywords'] = temp_list

In [15]:
df.head()

,show_id,title,cast,rating,listed_in,description,keywords
0,81145628,Norm of the North: King Sized Adventure,"[Alan Marriott, Andrew Toth, Brian Dobson]",TV-PG,"[Children, Family, Movies, Comedies]",Before planning an awesome wedding for his gra...,"[grandfather, polar, bear, king, must, take, b..."
1,80117401,Jandino: Whatever it Takes,[Jandino Asporaat],TV-MA,"[Stand-Up, Comedy]",Jandino Asporaat riffs on the challenges of ra...,"[rousing, rendition, sex, serenades, comedy, s..."
2,70234439,Transformers Prime,"[Peter Cullen, Sumalee Montano, Frank Welker]",TV-Y7-FV,"[Kids, TV]","With the help of three human allies, the Autob...","[autobots, help, decepticons, leader, onslaugh..."
3,80058654,Transformers: Robots in Disguise,"[Will Friedle, Darren Criss, Constance Zimmer]",TV-Y7,"[Kids, TV]",When a prison ship crash unleashes hundreds of...,"[earth, prison, ship, crash, unleashes, hundre..."
4,80125979,#realityhigh,"[Nesta Cooper, Kate Walsh, John Michael Higgins]",TV-14,[Comedies],When nerdy high schooler Dani finally attracts...,"[longtime, crush, ex, nerdy, high, schooler, d..."


In [0]:
# dropping the Plot column
df.drop(columns = ['description'], inplace = True)

In [17]:
df.set_index('title', inplace = True)
df.head()

,show_id,cast,rating,listed_in,keywords
title,,,,,
Norm of the North: King Sized Adventure,81145628,"[Alan Marriott, Andrew Toth, Brian Dobson]",TV-PG,"[Children, Family, Movies, Comedies]","[grandfather, polar, bear, king, must, take, b..."
Jandino: Whatever it Takes,80117401,[Jandino Asporaat],TV-MA,"[Stand-Up, Comedy]","[rousing, rendition, sex, serenades, comedy, s..."
Transformers Prime,70234439,"[Peter Cullen, Sumalee Montano, Frank Welker]",TV-Y7-FV,"[Kids, TV]","[autobots, help, decepticons, leader, onslaugh..."
Transformers: Robots in Disguise,80058654,"[Will Friedle, Darren Criss, Constance Zimmer]",TV-Y7,"[Kids, TV]","[earth, prison, ship, crash, unleashes, hundre..."
#realityhigh,80125979,"[Nesta Cooper, Kate Walsh, John Michael Higgins]",TV-14,[Comedies],"[longtime, crush, ex, nerdy, high, schooler, d..."


In [0]:
columns = df.columns
words = ''
# add each keyword to be one sentence
for index, row in df.iterrows():
  for col in columns:
    if col != 'show_id' and col != 'rating':
      words = words + ' '.join(row[col]) + ' '
  df['bag_of_words'] = words

# df.head()
df.drop(columns = [col for col in df.columns if col != 'bag_of_words'], inplace = True)

In [0]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

In [21]:
type(count_matrix)

scipy.sparse.csr.csr_matrix

In [22]:
count_matrix.shape

(5657, 24985)

In [0]:
count_matrix2 = count_matrix.toarray() # convert to array so can be used by Tensorflow

In [24]:
count_matrix2

array([[22,  2,  1, ...,  1,  1,  1],
       [22,  2,  1, ...,  1,  1,  1],
       [22,  2,  1, ...,  1,  1,  1],
       ...,
       [22,  2,  1, ...,  1,  1,  1],
       [22,  2,  1, ...,  1,  1,  1],
       [22,  2,  1, ...,  1,  1,  1]])

In [0]:
input_shape = (count_matrix2.shape[1], 1)
input_tensor = tf.keras.Input(shape=(input_shape))

In [0]:
# generating the cosine similarity matrix
model = tf.keras.Model(input_tensor, input_tensor)
model.compile(
    'sgd',
    loss='mse',
    metrics=[tf.keras.metrics.CosineSimilarity(axis=1)])

In [29]:
history = model.fit(count_matrix2, count_matrix2)

177/177 [==============================] - 2s 9ms/step - loss: 0.0000e+00 - cosine_similarity: 1.0000


In [0]:
history.history

{'cosine_similarity': [1.000001072883606], 'loss': [0.0]}

## the problem that I faced is I can not continue because I dont know how to make matrix of consine similarity using Tensorflow. So I stucked here!

In [0]:
m = tf.keras.metrics.CosineSimilarity(axis=1)

In [32]:
m.update_state(count_matrix2, count_matrix2)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=11314.0>

In [0]:
cosine_sim

<tf.Tensor: shape=(), dtype=float32, numpy=1.0000004>

In [0]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)

In [0]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [0]:
recommendations('Transformers')

IndexError: ignored